In [1]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pylab as plt
import pandas as pd
import sqlite3

In [31]:
db = "./data/houses.db.20170104"
price_csv_file = "./data/price.csv"
con = sqlite3.connect(db)
query = """
SELECT 
"链家编号", "标题", "总价", "建筑面积", "经度", "纬度"
FROM houses;
"""
df = pd.read_sql(query, con)
df["单价"] = df["总价"] / df["建筑面积"]
point_pos = 10000.0
fn = lambda x: int(x*point_pos+0.5)/point_pos
df["经度_集中"] = df["经度"].map(fn)
df["纬度_集中"] = df["纬度"].map(fn)
df_out = df[["经度_集中", "纬度_集中", "单价"]]
df_out["单价"].groupby([df_out["经度_集中"], df_out["纬度_集中"]]).mean().to_csv(price_csv_file)
df.describe()

,总价,建筑面积,经度,纬度,单价,经度_集中,纬度_集中
count,3282.000000,3282.000000,3282.000000,3282.000000,3282.000000,3282.000000,3282.000000
mean,235.904723,85.957233,113.328171,23.100288,2.579150,113.328172,23.100289
std,226.689264,40.783317,0.069812,0.073448,1.387674,0.069811,0.073447
min,30.000000,12.500000,113.194771,22.910274,0.656455,113.194800,22.910300
25%,110.000000,60.180000,113.281515,23.075056,1.630721,113.281525,23.075100
50%,160.000000,78.420000,113.314907,23.110019,2.145474,113.314900,23.110000
75%,280.000000,100.000000,113.361064,23.142768,3.069307,113.361100,23.142800
max,3400.000000,493.910000,113.610963,23.463009,10.810811,113.611000,23.463000


In [21]:
l_lat, u_lat = 22.7, 23.6
l_lon, u_lon = 113.0, 113.8
fig, axes = plt.subplots()
bmap = Basemap(ax=axes, projection="stere",
               lon_0=(l_lon+u_lon)/2,
               lat_0=(l_lat+u_lat)/2,
               llcrnrlat=l_lat, urcrnrlat=u_lat,
               llcrnrlon=u_lon, urcrnrlon=u_lon,
               resolution="f")
bmap.drawcoastlines()

x, y = bmap(df["经度"], df["纬度"])
#bmap.plot(x, y, 'k.', alpha=0.5)

SystemError: <class 'RuntimeError'> returned a result with an error set